In [ ]:
#https://github.com/nicharuc/Collocations/blob/master/Collocations.ipynb

In [17]:
import string

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [4]:
dat = pd.read_csv("./output_scrape/output_2019.csv")

In [5]:
dat.head()

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,2019-11-15 07:08:46,os_alexsm,warpling,0,0,0,Not sure if you have workflows that this appli...,NaN,@zapier,NaN,1195237188893495296,https://twitter.com/os_alexsm/status/119523718...
1,2019-11-15 07:07:42,Contextify1,Contextify1,0,0,0,Agree with the sentiment that it is rare to fi...,NaN,NaN,NaN,1195236919602364416,https://twitter.com/Contextify1/status/1195236...
2,2019-11-15 07:02:20,benvanessen,theJeremySchmit,0,0,0,@tferriss Curious to know if you have any Zapi...,NaN,@tferriss,NaN,1195235566515830786,https://twitter.com/benvanessen/status/1195235...
3,2019-11-15 06:58:01,renewhr,NaN,0,0,0,3 in 4 U.S. Knowledge Workers Would Be Willing...,NaN,NaN,NaN,1195234483286315008,https://twitter.com/renewhr/status/11952344832...
4,2019-11-15 06:46:25,smsads07,AndrewJDavison,0,0,0,OK will try this. Thanks,NaN,NaN,NaN,1195231562331303937,https://twitter.com/smsads07/status/1195231562...


In [6]:
dat["text"].head(10)

0    Not sure if you have workflows that this appli...
1    Agree with the sentiment that it is rare to fi...
2    @tferriss Curious to know if you have any Zapi...
3    3 in 4 U.S. Knowledge Workers Would Be Willing...
4                             OK will try this. Thanks
5    The Zapier Blog: 5 things retailers should do ...
6    MSPs delivering Zapier to small businesses acr...
7    We used over a dozen productivity apps includi...
8    What to do after Black Friday & Cyber Monday: ...
9    Tomorrow on my weekly Zoom in with Seth David ...
Name: text, dtype: object

In [7]:
from polyglot.detect import Detector

In [8]:
import re

# Count words in text. Preprocessing for filtering out text of word count = 1 so language detector can work
dat["num_words"] = dat["text"].apply(lambda x: len(re.findall(r'\w+', str(x))))

In [9]:
# Drop all rows where text is only 1 word
dat1 = dat[dat["num_words"] >1]

In [10]:
# dat1 reset index
dat1 = dat1.reset_index().drop(["index"], axis = 1)


In [11]:
import warnings
warnings.filterwarnings(action = "ignore")

from polyglot.detect import Detector

dat1["detector"] = dat1["text"].apply(lambda x: Detector(x, quiet = True))

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [12]:
# Extract language and confidence level from Detector objects
# https://stackoverflow.com/questions/51503199/how-to-apply-polyglot-detector-function-to-dataframe

import icu
dat1["language"] = dat1["detector"].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
dat1["confidence"] = dat1["detector"].apply(lambda x: x.language.confidence)

In [13]:
dat2 = dat1[dat1["language"] == "English"].reset_index().drop(columns = "index")

In [14]:
dat2.shape

(26660, 16)

In [15]:
import re 

# Check if there is a question mark
dat2["question_mark"] = dat2["text"].apply(lambda x: len(re.findall(r".+\?.*", x)) >= 1 )

In [16]:
for i in dat2["text"].head():
    print(i)
    print("\n")

Not sure if you have workflows that this applies to, but @zapier is my jam. Saves me SOOOOOO much time


Agree with the sentiment that it is rare to find a good, large, bootstrapped consumer business. Any sufficiently large TAM will attract competitors, many of whom have no qualms with soliciting investment (e.g. speculation that 1Pass's round was spurred by Dashlane's before it)


@tferriss Curious to know if you have any Zapier pro tips! What do you automate?


3 in 4 U.S. Knowledge Workers Would Be Willing to Quit to Work From Home, Zapier Report Finds https://www.hrtechnologist.com/news/mobile-workforce/3-in-4-us-knowledge-workers-would-be-willing-to-quit-to-work-from-home-zapier-report-finds/?utm_source=dlvr.it&utm_medium=twitter&utm_campaign=rss_subscribers …


OK will try this. Thanks




## Punctuation, URL and text removal/replacements but without stopwords removal

In [518]:
import re
import string
from nltk.tokenize import word_tokenize

# Function for cleaning text by removing punctuation only
def clean_text(text):
    # remove all urls as identified by "http" and "www". 
    # "\S+" removes all non-whitespace characters as part of the url until the end of url as defined by whitespace
    # https://stackoverflow.com/questions/24399820/expression-to-remove-url-links-from-twitter-tweet/24399874
    regex_url = re.compile(r"http\S+"+"|"
                           +"www\S+"+"|"
                          +"\S+\.com\S+")
    dirty_text = regex_url.sub(" ", str(text).lower())
    
    # Replace "i'm" with "i am"
    regex_im = re.compile(r"i'm ")
    dirty_text = regex_im.sub("i am ", str(dirty_text))
    
    # Replace "i'll" with "i will"
    regex_im = re.compile(r"i'll ")
    dirty_text = regex_im.sub("i will ", str(dirty_text))
    
    # Replace "let's" with "let us"
    regex_lets = re.compile(r"let's ")
    dirty_text = regex_lets.sub("let us ", str(dirty_text))
    
    # Replace "it's" with "it is"
    regex_its = re.compile(r"it's ")
    dirty_text = regex_its.sub("it is ", str(dirty_text))
    
    # Replace "there's" with "there is"
    regex_theres = re.compile(r"there's ")
    dirty_text = regex_theres.sub("there is ", str(dirty_text))
    
    # Replace "don't" with "do not"
    regex_dont = re.compile(r"don't ")
    dirty_text = regex_dont.sub("do not ", str(dirty_text))
    
    # Replace digits and digits with one instance of comma formatting
    regex_digits = re.compile(r"\d*[,]+\d*[+]?"+"|"+
                              "\d+")
    dirty_text = regex_digits.sub(" ", str(dirty_text))
    
    # Replace twitter tags
    regex_twittertags = re.compile(r"#\S+")
    dirty_text = regex_twittertags.sub(" ", str(dirty_text))
    
     # Replace twitter handles
    regex_twitterhandles = re.compile(r"@\S+")
    dirty_text = regex_twitterhandles.sub(" ", str(dirty_text))
    
    # remove punc except new lines "\n"
    regex_punct = re.compile('['+ re.escape(string.punctuation) + '\\r\\t]')
    nopunct = regex_punct.sub(" ", str(dirty_text))
    
    tokens = word_tokenize(nopunct.lower())
    
    return tokens

In [519]:
# Combine all list of tokens into one list, but segment between with new line escape "\n" to separate out tweet 
# (otherwise introduce false signal between last word of each tweet and first word of the next tweet for bigrams)
list_str = list(dat2["text"])
list_str_tot = "\n".join(list_str)

In [520]:
clean_tokens = clean_text(list_str_tot)

In [441]:
import nltk

# Bigrams finder
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean_tokens)

## 1. Frequency

In [442]:
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns = ["bigram", "freq"]).sort_values(by="freq", ascending = False)

In [443]:
from nltk.corpus import stopwords
#nltk.download('averaged_perceptron_tagger')

#get english stopwords
en_stopwords = set(stopwords.words('english'))
# add "zapier" and "..." to stopwords
en_stopwords.add("zapier")
en_stopwords.add("…")

stop_words_to_put_back = ["after", "any", "both", "between", "do","once","these","off",
                          #"on", "to", "and", "with","you",
                          "through","what", "when", "while","which", "where"]
for i in stop_words_to_put_back:
    en_stopwords.remove(i)

# Create function for filtering dataframe
def rightTypes(ngram):
    # remove stop words
    for word in ngram:
        if word in en_stopwords:
            return False
    
    # no conjunctions
    non_acceptable_types = ('CONJ')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in non_acceptable_types or tags[1][1] in non_acceptable_types:
        return False
    else:
        return True

In [444]:
clean_bigramFreqTable = bigramFreqTable[bigramFreqTable["bigram"].map(lambda x: rightTypes(x))].reset_index(drop = True)

In [445]:
clean_bigramFreqTable.head()

,bigram,freq
0,"(average, speed)",395
1,"(kph, mph)",395
2,"(elevation, gain)",395
3,"(metres, feet)",395
4,"(km, miles)",395


## 2. PMI

In [480]:
# Bigrams finder
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean_tokens)

# Filter out bigram frequencies
bigramFinder.apply_freq_filter(10)

In [481]:
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [483]:
bigramPMITable.head()

,bigram,PMI
0,"(dell, boomi)",15.089932
1,"(jenny, bloom)",15.089932
2,"(jessica, greene)",14.731478
3,"(storyteller, pairs)",14.731478
4,"(marcus, blankenship)",14.731478


In [485]:
bigramPMITable.query("PMI <= 10 & PMI >= 5").head()

,bigram,PMI
237,"(text, analysis)",9.993529
238,"(big, fan)",9.986367
239,"(life, balance)",9.983221
240,"(facebook, messenger)",9.979471
241,"(rbcpartner, ”)",9.973921


In [488]:
clean_bigramPMITable = bigramPMITable[bigramPMITable["bigram"].map(lambda x: rightTypes(x))].reset_index(drop = True)

In [489]:
clean_bigramPMITable 

,bigram,PMI
0,"(dell, boomi)",15.089932
1,"(jenny, bloom)",15.089932
2,"(jessica, greene)",14.731478
3,"(storyteller, pairs)",14.731478
4,"(marcus, blankenship)",14.731478
5,"(highlight, reel)",14.620447
6,"(keyword, extraction)",14.468444
7,"(optinmonster, trigger…)",14.430309
8,"(decision, matrix)",14.352967
9,"(reel, delivered)",14.330940


## 3. T-test

In [490]:
# Reset Bigrams finder due to previous filter on bigramFinder
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean_tokens)

# Filter out bigram frequencies
bigramFinder.apply_freq_filter(10)

In [491]:
bigramTtestTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)

In [492]:
clean_bigramTtestTable = bigramTtestTable[bigramTtestTable["bigram"].map(lambda x: rightTypes(x))].reset_index(drop = True)

In [493]:
clean_bigramTtestTable

,bigram,t
0,"(kph, mph)",19.860177
1,"(metres, feet)",19.860141
2,"(km, miles)",19.859626
3,"(elevation, gain)",19.859264
4,"(average, speed)",19.858413
5,"(feet, link)",19.668413
6,"(done, km)",19.541603
7,"(ride, done)",19.439279
8,"(google, sheets)",18.382836
9,"(social, media)",13.666831


## 4. chi-square

In [494]:
# Reset Bigrams finder due to previous filter on bigramFinder
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean_tokens)

# Filter out bigram frequencies
bigramFinder.apply_freq_filter(10)

In [495]:
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [500]:
clean_bigramChiTable = bigramChiTable[bigramChiTable["bigram"].map(lambda x: rightTypes(x))].reset_index(drop = True)

In [501]:
clean_bigramChiTable.head(10)

,bigram,chi-sq
0,"(kph, mph)",544060.000000
1,"(mlsp, funnelizer)",544060.000000
2,"(metres, feet)",542685.112912
3,"(km, miles)",524024.242549
4,"(elevation, gain)",511651.947203
5,"(traction, conf)",511254.230539
6,"(axis, lms)",492395.154659
7,"(marcus, blankenship)",489652.199940
8,"(average, speed)",484747.113983
9,"(san, francisco)",461622.424024


## 5. Likelihood Ratio

In [498]:
# Reset Bigrams finder due to previous filter on bigramFinder
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean_tokens)

# Filter out bigram frequencies
bigramFinder.apply_freq_filter(10)

In [499]:
bigramLikeTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [502]:
clean_bigramLikeTable = bigramLikeTable[bigramLikeTable["bigram"].map(lambda x: rightTypes(x))].reset_index(drop = True)

In [503]:
clean_bigramLikeTable.head(10)

,bigram,likelihood ratio
0,"(kph, mph)",6499.777100
1,"(metres, feet)",6485.816799
2,"(km, miles)",6356.256445
3,"(elevation, gain)",6310.226688
4,"(average, speed)",6141.220548
5,"(feet, link)",5221.247496
6,"(done, km)",5130.771659
7,"(ride, done)",5073.140486
8,"(google, sheets)",3737.421268
9,"(social, media)",2859.644954


## Bigrams Comparison

In [515]:
bigramsCompare = pd.DataFrame([clean_bigramFreqTable["bigram"][:100].values, clean_bigramPMITable["bigram"][:100].values, clean_bigramTtestTable["bigram"][:100].values, clean_bigramChiTable["bigram"][:100].values, clean_bigramLikeTable["bigram"][:100].values]).T

In [516]:
bigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test', 'Likeihood Ratio Test With Filter']

In [517]:
bigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test,Likeihood Ratio Test With Filter
0,"(average, speed)","(dell, boomi)","(kph, mph)","(kph, mph)","(kph, mph)"
1,"(kph, mph)","(jenny, bloom)","(metres, feet)","(mlsp, funnelizer)","(metres, feet)"
2,"(elevation, gain)","(jessica, greene)","(km, miles)","(metres, feet)","(km, miles)"
3,"(metres, feet)","(storyteller, pairs)","(elevation, gain)","(km, miles)","(elevation, gain)"
4,"(km, miles)","(marcus, blankenship)","(average, speed)","(elevation, gain)","(average, speed)"
5,"(feet, link)","(highlight, reel)","(feet, link)","(traction, conf)","(feet, link)"
6,"(done, km)","(keyword, extraction)","(done, km)","(axis, lms)","(done, km)"
7,"(ride, done)","(optinmonster, trigger…)","(ride, done)","(marcus, blankenship)","(ride, done)"
8,"(google, sheets)","(decision, matrix)","(google, sheets)","(average, speed)","(google, sheets)"
9,"(new, integration)","(reel, delivered)","(social, media)","(san, francisco)","(social, media)"
